# Imports

In [ ]:
import json
from pathlib import Path

# Go up from notebooks/ → code/
PROJECT_ROOT = Path.cwd().parent

CONFIG_PATH = PROJECT_ROOT / "param_config.json"
with open(CONFIG_PATH, "r") as f:
    param_config = json.load(f)

In [ ]:
import sys, os
sys.path.append(os.path.abspath(".."))

import pandas as pd
from gpcam.gp_optimizer import GPOptimizer

#convenience imports for analysis
import source_files.ExperimentalFunctions as EF
import source_files.UtilityFunctions as UF
import source_files.ConvergenceTesting as CT
import source_files.InitializationFunctions as IF

In [ ]:
UF.set_FH_prior_config(param_config)

# Code Execution

In [ ]:
import numpy as np

np_RNG_seed = param_config["general_parameters"]["np_RNG_seed"]

if isinstance(np_RNG_seed, int):
    np.random.seed(np_RNG_seed)

In [ ]:
#read in the raw measurements
raw_measurement_df = pd.read_excel(r"../../data/raw_measurements.xlsx")

In [ ]:
cloudiness_params = param_config["experimental_parameters"]["cloudiness_params"]
a_value_noise = param_config["general_parameters"]["a_value_noise"]
noise_floor = param_config["general_parameters"]["floor_noise"]

raw_measurement_df[["cloudiness", "variance_single"]] = raw_measurement_df.apply(
    lambda row: pd.Series(EF.get_ind_cloudiness_score_and_variance(row, cloudiness_params, noise_floor, a_value_noise)),
    axis=1
)


In [ ]:
df_clean = EF.collapse_all_measurements(raw_measurement_df, noise_floor = noise_floor)

In [ ]:
xT_array = df_clean[["x", "T"]].to_numpy()
measurement_array = df_clean["measurement"].to_numpy()
variance_array = df_clean["variance"].to_numpy()

In [ ]:
#final HPs from campaign provided below as initialization example
sample_init_hyperparameters = np.array([0.0643, 0.119, 2.629, 0.0217, -8.219])

#composition, temperature parameter bounds, useful for decision policy to generate new candidate points
parameter_bounds = [
    param_config["general_parameters"]["parameter_bounds"]["composition"],
    param_config["general_parameters"]["parameter_bounds"]["temperature_C"]
]


FH_HPs = param_config["general_parameters"]["hyperparameters"]
FH_HP_names = FH_HPs.keys()

#boundaries for each of the hyperparameters, useful for training
hyperparameter_bounds = [FH_HPs[k]["bounds"] for k in FH_HP_names]

#Initialize the GP using the GPOptimizer constructor from gpcam.gp_optimizer
my_GP_opt = GPOptimizer(
    x_data = xT_array,
    y_data = measurement_array, 
    init_hyperparameters = sample_init_hyperparameters, 
    noise_variances = variance_array,
    gp_kernel_function = IF.custom_kernel,
    gp_mean_function = UF.flory_huggins_prior_mean
)

In [ ]:
curr_optimizer_data = my_GP_opt.get_data()

for key, val in curr_optimizer_data.items():
    print('--'*20)
    print(f"{key}: {val}")